# Uzbek Emotion Classifier 

In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from memory_profiler import profile
import psutil
import logging

# Set up logging
logging.basicConfig(filename='memory_usage.log', level=logging.INFO)

# Load the CSV files
train_df = pd.read_csv('https://raw.githubusercontent.com/AkmalMir/DataScience/main/dataset/emotion_dataset_uzbek_train.csv')
val_df = pd.read_csv('https://raw.githubusercontent.com/AkmalMir/DataScience/main/dataset/emotion_dataset_uzbek_validation.csv')

# Load the Uzbek RoBERTa tokenizer and model
model_name = 'tahrirchi/tahrirchi-bert-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Tokenize the dataset
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_df['uzbek_text'].tolist())
val_encodings = tokenize_function(val_df['uzbek_text'].tolist())

# Create PyTorch datasets
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, train_df['label'].tolist())
val_dataset = EmotionDataset(val_encodings, val_df['label'].tolist())

# Define the metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted")
    }

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

# Function to get GPU memory usage
def get_gpu_memory_usage():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**2  # Convert to MB
    return 0

# Function to get CPU memory usage
def get_cpu_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024**2  # Convert to MB

# Modify the Trainer class to log memory usage
class MemoryTracker(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.step_count = 0
        self.print_frequency = 100  # Print every 100 steps

    def training_step(self, model, inputs):
        self.step_count += 1
        
        if self.step_count % self.print_frequency == 0:
            cpu_mem_before = get_cpu_memory_usage()
            gpu_mem_before = get_gpu_memory_usage()
            
            loss = super().training_step(model, inputs)
            
            cpu_mem_after = get_cpu_memory_usage()
            gpu_mem_after = get_gpu_memory_usage()
            
            mem_info = f"Step {self.step_count} - CPU Memory: {cpu_mem_after - cpu_mem_before:.2f} MB, GPU Memory: {gpu_mem_after - gpu_mem_before:.2f} MB"
            print(mem_info)
            logging.info(mem_info)
            
            return loss
        else:
            return super().training_step(model, inputs)

# Profile the training process
@profile
def train_model():
    global trainer
    trainer = MemoryTracker(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()

# Function for emotion prediction
def predict_emotion(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()
    emotion_labels = ["sadness", "joy", "love", "anger", "fear", "surprise"]
    return emotion_labels[predicted_class]

# Profile the inference process
@profile
def run_inference():
    test_text = "Men bugun juda xursandman!"  # "I'm very happy today!" in Uzbek
    predicted_emotion = predict_emotion(test_text)
    print(f"Text: '{test_text}'")
    print(f"Predicted emotion: {predicted_emotion}")

# Main execution
if __name__ == "__main__":
    print("Starting training...")
    train_model()
    
    print("\nEvaluating the model...")
    eval_results = trainer.evaluate()
    print(f"Evaluation results: {eval_results}")

    print("\nSaving the model...")
    model.save_pretrained("./emotion_classifier_model")
    tokenizer.save_pretrained("./emotion_classifier_tokenizer")

    print("\nRunning inference...")
    run_inference()

    print("\nComprehensive evaluation on the validation set...")
    predictions = trainer.predict(val_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    labels = val_df['label'].tolist()

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")

    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation Recall: {recall:.4f}")

    print("\nGenerating confusion matrix...")
    cm = confusion_matrix(labels, preds)
    emotion_labels = ["sadness", "joy", "love", "anger", "fear", "surprise"]
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotion_labels, yticklabels=emotion_labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()

    print("\nClassification Report:")
    print(classification_report(labels, preds, target_names=emotion_labels))

    print("\nMemory usage analysis complete. Check 'memory_usage.log' for detailed logs.")

2024-07-03 07:29:42.371165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 07:29:42.371288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 07:29:42.475748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/491k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/288k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tahrirchi/tahrirchi-bert-small and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...
ERROR: Could not find file /tmp/ipykernel_23/3278508086.py


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.992900,0.765646,0.734000,0.730950,0.737739,0.734000
2,0.599600,0.737609,0.746000,0.746912,0.755652,0.746000
3,0.307700,0.747007,0.760000,0.758109,0.758758,0.760000


Step 100 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 200 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 300 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 400 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 500 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 600 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 700 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 800 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 900 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1000 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1100 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1200 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1300 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1400 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1500 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1600 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1700 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1800 - CPU Memory: 0.00 MB, GPU Memory: 261.64 MB
Step 1900 - CPU Mem

Evaluation results: {'eval_loss': 0.7376086711883545, 'eval_accuracy': 0.746, 'eval_f1': 0.7469116752076659, 'eval_precision': 0.7556520632329997, 'eval_recall': 0.746, 'eval_runtime': 3.8435, 'eval_samples_per_second': 520.36, 'eval_steps_per_second': 8.326, 'epoch': 3.0}

Saving the model...

Running inference...
ERROR: Could not find file /tmp/ipykernel_23/3278508086.py
Text: 'Men bugun juda xursandman!'
Predicted emotion: joy

Comprehensive evaluation on the validation set...
Validation Accuracy: 0.7460
Validation F1 Score: 0.7469
Validation Precision: 0.7557
Validation Recall: 0.7460

Generating confusion matrix...

Classification Report:
              precision    recall  f1-score   support

     sadness       0.77      0.81      0.79       550
         joy       0.86      0.78      0.82       704
        love       0.72      0.57      0.63       178
       anger       0.64      0.71      0.67       275
        fear       0.58      0.76      0.66       212
    surprise       0.73